In [1]:
import numpy as np
import pandas as pd
import progressbar as pb
import time
import warnings
import text_processing as tp
warnings.filterwarnings('ignore')
print('Loading the dataframe')
# data = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')
print('Load is complete')
# sample_size = 100

Loading en_core_web_lg
Loading the dataframe
Load is complete


In [2]:
def remove_nan(df):
    print('Remove NA')
    blanks = []
    print("Before removing the NaN:")
    print(df.isnull().sum())
    for x in df.itertuples():
        if type(x.short_desc)!=str: # detect the NaN
            blanks.append(x.Index)
        elif not x.short_desc: # detect empty string
            blanks.append(x.Index)
    df.drop(blanks,inplace=True)
    print("\nAfter removing the NaN:\n",df.isnull().sum())
    
# remove_nan(data)

In [3]:
# Preprocessing the short_desc
# processed_data_df = tp.text_preprocessing(data,"short_desc")
# processed_data_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/processed_data_df.csv',index=False)
# print(processed_data_df.head())

In [2]:
processed_data_df = pd.read_csv('~/Desktop/Google-Drive/Colab Notebooks/processed_data_df.csv',sep=',')

In [5]:
# # Model-1: Similarity Score - Word2vec -------------------------------------------------------------------------------
# # Calculate similarity score Using word2vec
# print('Model-1: Similarity Score - Word2vec')
# sample_size = len(processed_data_df)
# start_time = time.time()

# import spacy
# print('Loading en_core_web_lg')
# nlp = spacy.load('en_core_web_lg')

# # Convert short_desc str to nlp format to fasten the computation process
# print('Convert short_desc str to nlp format to fasten the computation process')
# progress = pb.ProgressBar(maxval=sample_size).start()
# progvar = 0
# processed_data_nlp = []

# for tup in processed_data_df.itertuples():
#     processed_data_nlp.append((tup.id,tup.product,nlp(tup.short_desc)))
#     progress.update(progvar + 1)
#     progvar += 1
# progress = pb.ProgressBar(maxval=sample_size).finish()

# processed_data_nlp_df = pd.DataFrame(processed_data_nlp, columns=['id','product','short_desc'])
# processed_data_nlp_df.head()
# del processed_data_nlp

# # Calculate the cosine similarity score
# print('Calculate the similarity score')
# progress = pb.ProgressBar(maxval=sample_size).start()
# progvar = 0
# similarities_score_list = []

# for doc1 in processed_data_nlp_df.itertuples():
#     for doc2 in processed_data_nlp_df.itertuples():
#         if (doc1.id < doc2.id) and (doc1.product == doc2.product): # if two bug reports belong to the same product then check the similarity
#             similarity_score = doc1.short_desc.similarity(doc2.short_desc,)
#             similarities_score_list.append((doc1.id,doc2.id,similarity_score))
#     progress.update(progvar + 1)
#     progvar += 1
# progress = pb.ProgressBar(maxval=sample_size).finish()

# #convert to dataframe
# word2vec_similarities_score_df = pd.DataFrame(similarities_score_list, columns=['id1','id2','word2vec_score'] )
# del similarities_score_list
# word2vec_similarities_score_df = word2vec_similarities_score_df.reset_index(drop=True)
# word2vec_similarities_score_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/word2vec_similarities_score_df.csv',index=False)
# print("-Total- %s seconds ---" % (time.time() - start_time)) # show the time of process
# print('word2vec_similarities_score_df Shape:')
# print(word2vec_similarities_score_df.shape)

In [ ]:
# Model-2: Similarity Score - tf-idf -----------------------------------------------------------------------------------
print('Model-2: Similarity Score - tf-idf')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
start_time = time.time()
X_train = processed_data_df['short_desc']
sample_size = len(processed_data_df)
# Vectorization 
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set

# Calculate the cosine similarity score
print('Calculate the similarity score')
tfidf_cosine_similarities = linear_kernel(tfidf)
shape = tfidf_cosine_similarities.shape[0]
tfidf_cosine_similarities_list = []
progress = pb.ProgressBar(maxval=sample_size).start()
progvar = 0
for index1 in range(shape):
    for index2 in range(shape):
        if index2 > index1:
            tfidf_cosine_similarities_list.append([index1,index2,tfidf_cosine_similarities[index1,index2]])
    progress.update(progvar + 1)
    progvar += 1
progress = pb.ProgressBar(maxval=sample_size).finish()

#Conver to dataframe
tfidf_cosine_similarities_score_df = pd.DataFrame(tfidf_cosine_similarities_list, columns=['id1','id2','tfidf_score'])
del tfidf_cosine_similarities_list

# Convert index to id and Remove record of diff product
tp.index_to_id_remove_diff_product_score(tfidf_cosine_similarities_score_df, data)

tfidf_cosine_similarities_score_df = tfidf_cosine_similarities_score_df.reset_index(drop=True)
tfidf_cosine_similarities_score_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/tfidf_cosine_similarities_score_df.csv',index=False)
print("-Total- %s seconds ---" % (time.time() - start_time)) # show the time of process
print('tfidf_cosine_similarities_score_df Shape:')
print(tfidf_cosine_similarities_score_df.shape)

Model-2: Similarity Score - tf-idf
Calculate the similarity score


  3% (1679 of 43991) |                   | Elapsed Time: 0:01:13 ETA:   0:44:42

In [ ]:
# Model-3: Similarity Score - BM24F ------------------------------------------------------------------------------------
print('Model-3: Similarity Score - BM24F')
from rank_bm25 import BM25Okapi
progress = pb.ProgressBar(maxval=sample_size).start()
progvar = 0
start_time = time.time()

# preprocess - tokenize the short_desc to token
print('preprocess - tokenize the short_desc to token')
processed_corpus_list = []

for x in processed_data_df.itertuples():
    short_desc_splited = x.short_desc.split(" ")
    processed_corpus_list.append(short_desc_splited)
    
# Create a MB24 Object with the corpus
bm25 = BM25Okapi(processed_corpus_list)

# Calculate the similarity score for all bug reports
BM24_similarity_score_list = []

for x in processed_data_df.itertuples():
    query = processed_corpus_list[x.Index]
    doc_scores = bm25.get_scores(query)
    BM24_similarity_score_list.append([x.Index,doc_scores])

# Re-formating Matrix to List
print('Reformating Matrix to List')
shape = doc_scores.shape[0]
BM24_similarity_score_list_2 = []

for index1 in range(shape):
    for index2 in range(shape):
        if index2 > index1:
            BM24_similarity_score_list_2.append([index1,index2,BM24_similarity_score_list[index1][1][index2]])
    progress.update(progvar + 1)
    progvar += 1
progress = pb.ProgressBar(maxval=sample_size).finish()

del BM24_similarity_score_list

# Conver to dataframe
BM24_similarity_score_sort_df = pd.DataFrame(BM24_similarity_score_list_2, columns=['id1','id2','BM24_score'])
del BM24_similarity_score_list_2

# Convert index to id and Remove record of diff product
tp.index_to_id_remove_diff_product_score(BM24_similarity_score_sort_df,data)

BM24_similarity_score_sort_df = BM24_similarity_score_sort_df.reset_index(drop=True)
BM24_similarity_score_sort_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/BM24_similarity_score_sort_df.csv',index=False)
print("-Total- %s seconds ---" % (time.time() - start_time)) # show the time of process
print('BM24_similarity_score_sort_df Shape:')
print(BM24_similarity_score_sort_df.shape)

In [ ]:
# Assessment the Model Accuracy ----------------------------------------------------------------------------------------
print('Assessment the Model Accuracy')
duplicate_org = pd.read_csv('https://zenodo.org/record/400614/files/apache.relations.csv?download=1',sep=',')

In [ ]:
word2vec_df = tp.id_builder(word2vec_similarities_score_df)
tfidf_df = tp.id_builder(tfidf_cosine_similarities_score_df)
BM24_df = tp.id_builder(BM24_similarity_score_sort_df)

In [ ]:
total_score = pd.merge(word2vec_df, tfidf_df, on = 'ID')
total_score = pd.merge(total_score, BM24_df, on = 'ID')
total_score.head()

In [ ]:
df = pd.DataFrame(np.array([[2, 3], [2, 27],[2,31]]), columns=['id', 'dup'])
df

In [ ]:

progress = pb.ProgressBar(maxval=len(df)).start()
progvar = 0
start_time = time.time()
duplicate_score_table_list = []

for tup in df.itertuples():
    
    word2vec_score = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'word2vec_score'].array[0]
    tfidf_score = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'tfidf_score'].array[0]
    BM24_score  = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'BM24_score'].array[0]
    short_desc1 = data.loc[lambda df: df['id'] == tup.id,'short_desc'].array[0]
    short_desc2 = data.loc[lambda df: df['id'] == tup.dup,'short_desc'].array[0]
    
    duplicate_score_table_list.append([tup.id,tup.dup,word2vec_score,tfidf_score,BM24_score,short_desc1,short_desc2])

    progress.update(progvar + 1)
    progvar += 1
    
duplicate_score_table_df = pd.DataFrame(duplicate_score_table_list, columns=['id','dup','word2vec_score','tfidf_score','BM24_score','short_desc1','short_desc2'])
del duplicate_score_table_list
duplicate_score_table_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/duplicate_score_table_df.csv',index=False)
print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process
duplicate_score_table_df

In [ ]:

progress = pb.ProgressBar(maxval=len(df)).start()
progvar = 0
start_time = time.time()
duplicate_score_table_list = []

for tup in df.itertuples():
    
    word2vec_score = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'word2vec_score'].array[0]
    tfidf_score = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'tfidf_score'].array[0]
    BM24_score  = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'BM24_score'].array[0]
    short_desc1 = data.loc[lambda df: df['id'] == tup.id,'short_desc'].array[0]
    short_desc2 = data.loc[lambda df: df['id'] == tup.dup,'short_desc'].array[0]
    
    duplicate_score_table_list.append([tup.id,tup.dup,word2vec_score,tfidf_score,BM24_score,short_desc1,short_desc2])

    progress.update(progvar + 1)
    progvar += 1
    
duplicate_score_table_df = pd.DataFrame(duplicate_score_table_list, columns=['id','dup','word2vec_score','tfidf_score','BM24_score','short_desc1','short_desc2'])
del duplicate_score_table_list
duplicate_score_table_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/duplicate_score_table_df.csv',index=False)
print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process
duplicate_score_table_df

In [ ]:
df = pd.DataFrame(np.array([[2, 3], [2, 27],[2,31]]), columns=['id', 'dup'])
df

In [ ]:

progress = pb.ProgressBar(maxval=len(df)).start()
progvar = 0
start_time = time.time()
duplicate_score_table_list = []

for tup in df.itertuples():
    
    word2vec_score = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'word2vec_score'].array[0]
    tfidf_score = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'tfidf_score'].array[0]
    BM24_score  = total_score.loc[(total_score['id1'] == tup.id) & (total_score['id2'] == tup.dup),'BM24_score'].array[0]
    short_desc1 = data.loc[lambda df: df['id'] == tup.id,'short_desc'].array[0]
    short_desc2 = data.loc[lambda df: df['id'] == tup.dup,'short_desc'].array[0]
    
    duplicate_score_table_list.append([tup.id,tup.dup,word2vec_score,tfidf_score,BM24_score,short_desc1,short_desc2])

    progress.update(progvar + 1)
    progvar += 1
    
duplicate_score_table_df = pd.DataFrame(duplicate_score_table_list, columns=['id','dup','word2vec_score','tfidf_score','BM24_score','short_desc1','short_desc2'])
del duplicate_score_table_list
duplicate_score_table_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/duplicate_score_table_df.csv',index=False)
print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process
duplicate_score_table_df